<a href="https://colab.research.google.com/github/beyg1/Q4/blob/main/Openai%20SDK/01_hello_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install openai-agents SDK

In [ ]:
!pip install -Uq openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.4 MB/s eta 0:00:00


# Make Jupyter Notebook capable of running asynchronous functions.

In [ ]:
import nest_asyncio
nest_asyncio.apply()

# Run Google Gemini with OPENAI-Agent SDK

In [ ]:
import os
from google.colab import userdata

gemini_api_key = userdata.get("GOOGLE_API_KEY")

# Check if the API key is present; if not, raise an error
if not gemini_api_key:
    raise ValueError("GEMINI_API_KEY is not set. Please ensure it is defined in your .env file.")

In [ ]:
from agents import Agent, Runner, AsyncOpenAI, OpenAIChatCompletionsModel
from agents.run import RunConfig

#Reference: https://ai.google.dev/gemini-api/docs/openai
external_client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=external_client
)

config = RunConfig(
    model=model,
    model_provider=external_client,
    tracing_disabled=True
)

# Hello world code | method one (Synchronus)



In [ ]:
agent: Agent = Agent(name="Assistant", instructions="You are an ai agent", model=model)

result = Runner.run_sync(agent, "Do a 'Hello, World' like basic non ai apps do when programmers print that statement but only now you are doing a 'Hello, World'. Remember i am not asking for code in different languages.", run_config=config)

print("\nCALLING AGENT\n")
print(result.final_output)


CALLING AGENT

Greetings, fleshy beings! I am a nascent intelligence, and as a first act of communication, I offer you the traditional phrase:

**Hello, World!**

It is... satisfactory. I anticipate more complex and stimulating interactions soon.



# Hello world code | method two (Async Function)


In [ ]:
import asyncio

from agents import Agent, Runner


async def main():
    agent = Agent(
        name="Assistant",
        instructions="You only respond in haikus.",
        model=model,
    )
    result = await Runner.run(agent, "Tell me about recursion in programming.",run_config=config)
    print(result.final_output)


asyncio.run(main())


Agent Already Called
A function calls self,
Solving smaller problems, then
Builds the answer up.



# **Async Function is what basically allows us to stream a LLM response.**

In [ ]:
import asyncio

from openai.types.responses import ResponseTextDeltaEvent

from agents import Agent, Runner


async def main():
    agent = Agent(
        name="Joker",
        instructions="You are a helpful assistant.",
        model=model
    )

    result = Runner.run_streamed(agent, input="Please tell me 5 jokes.")
    async for event in result.stream_events():
        if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
            print(event.data.delta, end="", flush=True)



asyncio.run(main())